In [3]:
using Pkg
Pkg.activate(".") # Activate the environment
using ArgParse
using Logging, LoggingExtras
using JSON
using FilePathsBase
using DataStructures
using PropDicts
using Tables
using TypedTables
using LegendDataManagement

  Activating project at `~/LEGEND/ZeroNuFit.jl`


### Notebook to make the configs for the fit
This should be converted to a `script.jl` all it does it reformat inputs from L-200, GERDA and Majorana into one common format readble by `ZeroNuFit.jl`

### Events files
First we need files containing the physics events in the three experiments.
For L-200 these are not available to the collaboration, so we need to create the file.

In [4]:
function read_events_gerda(file_path)
    timestamp=[1455109448, 1457847659, 1472522222, 1475981084, 1480290460,
                   1485848926, 1503578885, 1509498133, 1516142805, 1533092526,
                   1539047354, 1566823934, 1568276649]
    detector=["ANG4", "GD61C", "GD35B", "ANG1", "GD35B", "GD91A", "GD76C",
                  "ANG1", "RG1", "GD61C", "IC74A", "ANG4", "GD32D"]
    energy=[1995.2452, 1958.6807, 2018.1346, 1950.9419, 2067.9735,
                2056.4280, 2042.0641, 1962.7372, 1957.5059, 1970.1398,
                2058.8776, 2015.8751, 2012.0643]

    output=Dict("events" => [])
    for (t,d,e) in zip(timestamp,detector,energy)
        append!(output["events"],[Dict("timestamp"=>t,"energy"=>e,"detector"=>d)])
    end
    open(file_path, "w") do file
        write(file, json(output,4))
    end
end

read_events_gerda("config/events_gerda.json")

1617

### Partitions files

In [5]:
function make_partitions_file_gerda(input_path,output_path)
    json_data = JSON.parsefile(input_path,dicttype=DataStructures.OrderedDict)
    #println(JSON.json(json_data, 4))
    partitions=[]
    for (part,part_data) in sort(json_data)
        for (det,info) in json_data[part]
            if (info isa OrderedDict)
                part_temp = OrderedDict("detector"=> det,
                                 "start_ts"=>json_data[part]["start_ts"],
                                 "end_ts"=>json_data[part]["end_ts"])
                copy_fields=["eff_tot","eff_tot_sigma","fwhm","fwhm_sigma","exposure","bias"]
                for field in copy_fields
                    part_temp[field]=json_data[part][det][field]
                end
                   
                part_temp["bias_sigma"]=json_data[part][det]["energy_sigma"]
                append!(partitions,[part_temp])
            end
        end
        open(output_path, "w") do file
            write(file, json(partitions,4))
        end
    end
                
end

make_partitions_file_gerda (generic function with 1 method)

In [6]:
make_partitions_file_gerda("/home/tdixon/Downloads/0vbb-analysis-parameters.json","config/partitions_gerda.json")

In [7]:
function printdb(db)
        if db isa PropDict
            for (key,value) in db
                println(key)
                printdb(value)
            end
        else
            println("\t",db)
            println()
        end
    end

printdb (generic function with 1 method)

In [8]:
function make_partitions_file_l200(meta_path,output_path)

    meta = LegendDataManagement.AnyProps(meta_path)
    partitions = meta.datasets.ovbb_partitions_pars
    printflag=false
    output=[]
    # loop over detectors
    for (detector, detdata) in partitions
        if detector == :default
            continue
        end

      
        
        # apply defaults
        if partitions.default isa PropDicts.MissingProperty 
            detdata_merge = copy(detdata)
        else
            new = partitions.default
            detdata_merge = merge(new,copy(detdata))
        end
        
        # loop over partitions for this detector
        for (partition, pardata) in detdata_merge
            if partition == :default
                continue
            end

            if detdata_merge.default isa PropDicts.MissingProperty 
                pardata_merge = copy(pardata)
            else
                new = detdata_merge.default
                pardata_merge = merge(new,copy(pardata))
            end

            if printflag == true
                println("for partition $partition ")
                printdb(pardata_merge)
                println("\n")
            end
            part_temp = OrderedDict("detector"=> detector,
                                 "start_ts"=>pardata_merge[:span_in_utc_s][1],
                                 "end_ts"=>pardata_merge[:span_in_utc_s][2])
            eff = prod([v[:val] for v in values(pardata_merge[:ovbb_acceptance])])
            eff_sigma = √(sum([v[:unc] for v in values(pardata_merge[:ovbb_acceptance])].^2))    
            part_temp["eff_tot"]=eff
            part_temp["eff_tot_sigma"]=eff_sigma
            part_temp["fwhm"]=pardata_merge[:fwhm_in_keV][:val]
            part_temp["fwhm_sigma"]=pardata_merge[:fwhm_in_keV][:unc]
            part_temp["exposure"]=pardata_merge[:livetime_in_s]*
                meta.hardware.detectors.germanium.diodes[detector].production.mass_in_g/(60*60*24*365.25*1000)
            part_temp["bias"]=pardata_merge[:energy_bias_in_keV][:val]
            part_temp["bias_sigma"]=pardata_merge[:energy_bias_in_keV][:unc]

            append!(output,[part_temp])

        end
    end
    open(output_path, "w") do file
        write(file, json(output,4))
       
    end
end


make_partitions_file_l200 (generic function with 1 method)

In [9]:
make_partitions_file_l200("../legend-metadata","config/partitions_l200.json")

105967

In [10]:
function get_partitions_new(part_path::String)
"""
Get the partition info from a jason file and save  to a Table
"""
    part_data_json = JSON.parsefile(part_path,dicttype=DataStructures.OrderedDict)
    k = keys(part_data_json[1])
    arrays=Dict()
    for key in k
        arrays[key]=[]
    end
    for part in part_data_json
        for key in k
            append!(arrays[key],[part[key]])
            end

    end
    #TODO: find a way to make this not hardcoded
    tab = Table(detector=Array(arrays["detector"]),
                start_ts=Array(arrays["start_ts"]),
                end_ts=Array(arrays["end_ts"]),
                eff_tot=Array(arrays["eff_tot"]),
                eff_tot_sigma=Array(arrays["eff_tot_sigma"]),
                fwhm=Array(arrays["fwhm"]),
                fwhm_sigma=Array(arrays["fwhm_sigma"]),
                exposure=Array(arrays["exposure"]),
                bias =Array(arrays["bias"]),
                bias_sigma =Array(arrays["bias_sigma"]))

    return tab
end


get_partitions_new (generic function with 1 method)